In [1]:
# TODO make a notebook with it
from transformers import AutoTokenizer, AutoModel
from pathlib import Path
import tensorflow as tf
import polars as pl
import os

#
from ebrec.utils._constants import (
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_SUBTITLE_COL,
    DEFAULT_LABELS_COL,
    DEFAULT_TITLE_COL,
    DEFAULT_USER_COL,
)

#
from ebrec.utils._behaviors import (
    create_binary_labels_column,
    sampling_strategy_wu2019,
    add_known_user_column,
    add_prediction_scores,
    truncate_history,
)
from ebrec.utils._articles import convert_text2encoding_with_transformers
from ebrec.utils._polars import concat_str_columns, slice_join_dataframes
from ebrec.utils._articles import create_article_id_to_value_mapping
from ebrec.utils._nlp import get_transformers_word_embeddings
from ebrec.utils._python import write_submission_file, rank_predictions_by_score


from ebrec.models.newsrec.model_config import hparams_npa
from ebrec.models.newsrec.dataloader import LSTURDataLoader
from ebrec.models.newsrec.npa import NPAModel
import numpy as np


# Clear session to reset the GPU state
tf.keras.backend.clear_session()


# Set memory growth to true for all GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


2024-06-24 14:25:04.734172: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-24 14:25:04.758508: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 14:25:04.758533: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 14:25:04.759388: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-24 14:25:04.763474: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-24 14:25:04.763836: I tensorflow/core/platform/cpu_feature_guard.cc:1

### Load dataset

In [2]:
data_base = Path("../../data/processed/")


df_train_articles = pl.scan_parquet(data_base / "train_articles.parquet")
df_test_articles = pl.scan_parquet(data_base / "test_articles.parquet")

df_train = pl.scan_parquet(data_base / "train.parquet").collect()
df_validation = pl.scan_parquet(data_base / "validation.parquet").collect()
df_test = pl.scan_parquet(data_base / "test.parquet").collect()

In [3]:
TRANSFORMER_MODEL_NAME = "FacebookAI/xlm-roberta-base"
TEXT_COLUMNS_TO_USE = [DEFAULT_SUBTITLE_COL, DEFAULT_TITLE_COL]
MAX_TITLE_LENGTH = 30

# LOAD HUGGINGFACE:
transformer_model = AutoModel.from_pretrained(TRANSFORMER_MODEL_NAME)
transformer_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

# We'll init the word embeddings using the
word2vec_embedding = get_transformers_word_embeddings(transformer_model)
#

df_articles = df_train_articles.collect()
df_articles_test = df_test_articles.collect()

df_articles, cat_cal = concat_str_columns(df_articles, columns=TEXT_COLUMNS_TO_USE)
df_articles, token_col_title = convert_text2encoding_with_transformers(
    df_articles, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH
)
# =>
article_mapping = create_article_id_to_value_mapping(
    df=df_articles, value_col=token_col_title
)
#
df_articles_test, cat_cal = concat_str_columns(df_articles_test, columns=TEXT_COLUMNS_TO_USE)
df_articles_test, token_col_title = convert_text2encoding_with_transformers(
    df_articles_test, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH
)
# =>
article_mapping_test = create_article_id_to_value_mapping(
    df=df_articles_test, value_col=token_col_title
)

In [4]:
df_validation.head(5)

shape: (5, 6)
┌─────────┬───────────────────┬───────────────────┬──────────────────┬───────────────┬─────────────┐
│ user_id ┆ article_id_fixed  ┆ article_ids_invie ┆ article_ids_clic ┆ impression_id ┆ labels      │
│ ---     ┆ ---               ┆ w                 ┆ ked              ┆ ---           ┆ ---         │
│ u32     ┆ list[i32]         ┆ ---               ┆ ---              ┆ u32           ┆ list[i8]    │
│         ┆                   ┆ list[i32]         ┆ list[i32]        ┆               ┆             │
╞═════════╪═══════════════════╪═══════════════════╪══════════════════╪═══════════════╪═════════════╡
│ 835650  ┆ [9780020,         ┆ [9785500,         ┆ [9786209]        ┆ 240732905     ┆ [0, 0, … 0] │
│         ┆ 9780039, …        ┆ 9773779, …        ┆                  ┆               ┆             │
│         ┆ 9777822]          ┆ 9782085]          ┆                  ┆               ┆             │
│ 2045557 ┆ [9777182,         ┆ [9786066,         ┆ [9785923]        ┆ 396809800     ┆ [0, 0, … 0] │
│         ┆ 9779641, …        ┆ 9786176, …        ┆                  ┆               ┆             │
│         ┆ 9778158]          ┆ 9785751]          ┆                  ┆               ┆             │
│ 1771895 ┆ [9776449,         ┆ [9788917,         ┆ [9788197]        ┆ 341847352     ┆ [0, 0, … 0] │
│         ┆ 9776385, …        ┆ 9786289, …        ┆                  ┆               ┆             │
│         ┆ 9779538]          ┆ 9782845]          ┆                  ┆               ┆             │
│ 1051979 ┆ [9777320,         ┆ [9785062,         ┆ [9786222]        ┆ 353866724     ┆ [0, 1, … 0] │
│         ┆ 9777320, …        ┆ 9786222, …        ┆                  ┆               ┆             │
│         ┆ 9778788]          ┆ 9786268]          ┆                  ┆               ┆             │
│ 2028365 ┆ [9777769,         ┆ [9782181,         ┆ [9782181]        ┆ 184176970     ┆ [1, 0, … 0] │
│         ┆ 9779737, …        ┆ 9784097, …        ┆                  ┆               ┆             │
│         ┆ 9779737]          ┆ 9785009]          ┆                  ┆               ┆             │
└─────────┴───────────────────┴───────────────────┴──────────────────┴───────────────┴─────────────┘

### Initiate dataloaders

In [5]:
BATCH_SIZE = 16
user_id_mapping = {user_id: i for i, user_id in enumerate(df_train[DEFAULT_USER_COL].unique())}

print("Initiating the dataloaders")
train_dataloader = LSTURDataLoader(
    user_id_mapping=user_id_mapping,
    behaviors=df_train,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=False,
    batch_size=BATCH_SIZE,
)
val_dataloader = LSTURDataLoader(
    user_id_mapping=user_id_mapping,
    behaviors=df_validation,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=True,
    batch_size=BATCH_SIZE,
)

test_dataloader = LSTURDataLoader(
    user_id_mapping=user_id_mapping,
    behaviors=df_test,
    article_dict=article_mapping_test,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=True,
    batch_size=BATCH_SIZE,
)

Initiating the dataloaders


In [6]:
from tensorflow.keras import mixed_precision
# Equivalent to the two lines above
mixed_precision.set_global_policy('mixed_float16')


The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [ ]:

# Your existing model setup and training code
MODEL_NAME = "NPA-monday-14-00"
LOG_DIR = f"./runs/{MODEL_NAME}"
MODEL_WEIGHTS = f"runs/{MODEL_NAME}/{MODEL_NAME}.weights.h5"

from tensorflow.keras.losses import CategoricalCrossentropy
HISTORY_SIZE = 10
# CALLBACKS
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=2)
modelcheckpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=MODEL_WEIGHTS, save_best_only=True, save_weights_only=True, verbose=1
)

hparams_npa.history_size = HISTORY_SIZE
npa_model = NPAModel(
    hparams=hparams_npa,
    word2vec_embedding=word2vec_embedding,
    seed=42,
)

# Build the model and scorer
model, scorer = npa_model._build_graph()

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=hparams_npa.learning_rate),
    loss=CategoricalCrossentropy(),
    metrics=['accuracy']
)

# Train the model
hist = model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=10,
    batch_size = 16,
    callbacks=[tensorboard_callback, early_stopping, modelcheckpoint],
    verbose = 1
)

# Load the best model weights
model.load_weights(filepath=MODEL_WEIGHTS)


Epoch 1/10
 29/147 [====>.........................] - ETA: 23:21 - loss: 1.6402 - accuracy: 0.2586

In [ ]:
HISTORY_SIZE = 10
# Your existing model setup and training code
MODEL_NAME = "NPA"
LOG_DIR = f"./runs/{MODEL_NAME}"
MODEL_WEIGHTS = f"runs/{MODEL_NAME}/{MODEL_NAME}.weights.h5"
from tensorflow.keras.losses import CategoricalCrossentropy
hparams_npa.history_size = HISTORY_SIZE
npa_model = NPAModel(
    hparams=hparams_npa,
    word2vec_embedding=word2vec_embedding,
    seed=42,
)

# Build the model and scorer
model, scorer = npa_model._build_graph()

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=hparams_npa.learning_rate),
    loss=CategoricalCrossentropy(),
    metrics=['accuracy']
)

# Initialize optimizer
dummy_data = next(iter(train_dataloader))
dummy_loss = model.train_on_batch(dummy_data[0], dummy_data[1])

# Load the best model weights
model.load_weights(filepath=MODEL_WEIGHTS)

In [ ]:
pred_validation = scorer.predict(val_dataloader)

df_validation = add_prediction_scores(df_validation, pred_validation.tolist()).pipe(
    add_known_user_column, known_users=df_train[DEFAULT_USER_COL]
)
df_validation.head(2)

from ebrec.evaluation import MetricEvaluator, AucScore, NdcgScore, MrrScore

metrics = MetricEvaluator(
    labels=df_validation["labels"].to_list(),
    predictions=df_validation["scores"].to_list(),
    metric_functions=[AucScore(), MrrScore(), NdcgScore(k=5), NdcgScore(k=10)],
)
metrics.evaluate()



